In [0]:
--=========================================================
-- Gold Opportunity Snapshot Table
--=========================================================
-- Purpose: Provide a comprehensive, wide table for each opportunity, joining key related dimensions.
--          including account, owner, creator, owner's manager, and aggregated line item data.
-- Grain: One row per opportunity_id.
--=========================================================

CREATE OR REFRESH LIVE TABLE gold_opportunity_snapshot (
  CONSTRAINT opportunity_id_not_null EXPECT (opportunity_id IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "Denormalized snapshot of opportunities, enriched with account, owner, creator, and line item aggregated data."
TBLPROPERTIES ("quality" = "gold")
AS
WITH opp_line_agg AS (
  -- Aggregate line item data first
  SELECT
    opportunity_id,
    SUM(total_price) AS total_line_item_amount,
    COUNT(*) AS count_line_items,
    COLLECT_LIST(product_id) AS list_of_product_ids -- Collect product IDs associated with the opportunity
  FROM live.silver_opportunity_product
  WHERE is_deleted = false -- Exclude deleted line items from aggregation
  GROUP BY opportunity_id
)
SELECT
  -- Opportunity Core Fields
  so.opportunity_id,
  so.account_id,
  so.owner_user_id,
  so.primary_contact_id,
  so.created_by_id,
  so.opportunity_type,
  so.stage_name,
  so.forecast_category,
  so.lead_source AS opportunity_lead_source, -- Renamed to avoid conflict with account lead source if joined
  so.amount,
  so.expected_amount,
  so.probability_percent,
  so.backlog_revenue,
  so.created_datetime AS opportunity_created_datetime,
  so.close_date,
  so.last_modified_datetime AS opportunity_last_modified_datetime,
  so.last_stage_change_datetime,
  so.is_closed,
  so.is_won,
  so.closed_lost_reason,
  so.opportunity_status,
  so.is_order_finalized,
  so.product_category,
  so.product_of_interest,
  so.industry AS opportunity_industry, -- Renamed for clarity if joined with account industry
  so.has_line_item,
  so.has_open_activity,
  so.has_overdue_task,
  so.is_deleted AS opportunity_is_deleted,

  -- Calculated Opportunity Fields
  CASE
    WHEN so.is_closed = true AND so.is_won = true THEN datediff(so.close_date, date(so.created_datetime))
    ELSE NULL
  END AS sales_cycle_days,
  datediff(current_date(), date(so.last_stage_change_datetime)) AS days_since_last_stage_change,
  CASE WHEN so.is_closed = true AND so.is_won = true THEN true ELSE false END AS is_closed_won,
  CASE WHEN so.is_closed = true AND so.is_won = false THEN true ELSE false END AS is_closed_lost,
  YEAR(so.close_date) || '-Q' || QUARTER(so.close_date) AS quarter_close_date,
  DATE_TRUNC('MONTH', so.close_date) AS month_close_date,
  DATE_TRUNC('MONTH', so.created_datetime) AS month_created_date,


  -- Account Fields (Joined)
  sa.account_name,
  sa.account_number,
  sa.account_type,
  sa.industry AS account_industry,
  sa.account_rating,
  sa.account_site,
  sa.account_ownership,
  sa.billing_city AS account_billing_city,
  sa.billing_state_province AS account_billing_state,
  sa.billing_country AS account_billing_country,
  sa.shipping_city AS account_shipping_city,
  sa.shipping_state_province AS account_shipping_state,
  sa.shipping_country AS account_shipping_country,
  sa.shipping_zip AS account_shipping_zip,
  sa.parent_account_id,
  sa.created_datetime AS account_created_datetime,
  sa.last_activity_date AS account_last_activity_date,
  sa.is_demo_center AS account_is_demo_center,
  sa.is_life_science_key_account AS account_is_life_science_key,
  sa.is_deleted AS account_is_deleted,

  -- Owner Fields (Joined)
  sou.full_name AS owner_full_name,
  sou.department AS owner_department,
  sou.manager_user_id AS owner_manager_user_id, -- Get the Owner's Manager ID
  sou.is_active AS owner_is_active,

  -- Owner's Manager Fields (Joined)
  som.full_name AS owner_manager_full_name, -- Get the Manager's full name
  som.department AS owner_manager_department, -- Get the Manager's department (if needed)
  som.is_active AS owner_manager_is_active, -- Get the Manager's active status (if needed)

  -- Creator Fields (Joined)
  scr.full_name AS created_by_full_name,
  scr.department AS created_by_department,
  scr.is_active AS created_by_is_active,

  -- Aggregated Line Item Fields (Joined)
  COALESCE(ola.total_line_item_amount, 0.0) AS total_line_item_amount, -- Default to 0 if no line items
  COALESCE(ola.count_line_items, 0) AS count_line_items,               -- Default to 0 if no line items
  ola.list_of_product_ids

FROM live.silver_opportunity so
LEFT JOIN live.silver_account sa ON so.account_id = sa.account_id
LEFT JOIN live.silver_user_dim sou ON so.owner_user_id = sou.user_id -- Join for Owner info
LEFT JOIN live.silver_user_dim scr ON so.created_by_id = scr.user_id -- Join for Creator info
LEFT JOIN live.silver_user_dim som ON sou.manager_user_id = som.user_id -- Join for Owner's Manager info using the manager_user_id from the owner's record
LEFT JOIN opp_line_agg ola ON so.opportunity_id = ola.opportunity_id;


In [0]:
--=========================================================
-- Gold Lead Funnel Table
--=========================================================
-- Purpose: Provide a comprehensive view of each lead, tracking its journey and enrichment.
-- Grain: One row per lead_id.
--=========================================================

CREATE OR REFRESH LIVE TABLE gold_lead_funnel (
  CONSTRAINT lead_id_not_null EXPECT (lead_id IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "Denormalized view of leads, enriched with owner and converted account information."
TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  -- Lead Core Fields
  sl.lead_id,
  sl.converted_account_id,
  sl.converted_opportunity_id,
  sl.companion_lead,
  sl.company_name,
  sl.lead_source,
  sl.lead_status,
  sl.industry AS lead_industry, -- Renamed for clarity
  sl.lead_type,
  sl.lead_record_type,
  sl.status_conversion,
  sl.city AS lead_city,
  sl.state_province AS lead_state_province,
  sl.country AS lead_country,
  sl.created_datetime AS lead_created_datetime,
  sl.converted_datetime,
  sl.last_activity_date AS lead_last_activity_date,
  sl.is_converted,
  sl.has_opted_out_email,
  sl.dead_reason,
  sl.is_key_account AS lead_is_key_account,
  sl.lead_score,
  sl.pardot_score,
  sl.mark_segmentation,
  sl.mass_spec_manufacturer,
  sl.mass_spec_type,
  -- sl.is_deleted AS lead_is_deleted -- Assuming this field exists if needed

  -- Calculated Lead Fields
  CASE
    WHEN sl.is_converted = true THEN datediff(date(sl.converted_datetime), date(sl.created_datetime))
    ELSE NULL
  END AS days_to_convert,
  datediff(current_date(), sl.last_activity_date) AS days_since_last_activity,
  CASE WHEN sl.is_converted = false AND sl.dead_reason IS NOT NULL THEN true ELSE false END AS is_dead, -- Basic logic, might refine based on status
  YEAR(sl.created_datetime) || '-Q' || QUARTER(sl.created_datetime) AS quarter_created_date,
  YEAR(sl.converted_datetime) || '-Q' || QUARTER(sl.converted_datetime) AS quarter_converted_date,
  DATE_TRUNC('MONTH', sl.created_datetime) AS month_created_date,
  DATE_TRUNC('MONTH', sl.converted_datetime) AS month_converted_date,


  -- Owner Fields (Joined) - Assuming Owner ID exists in silver_lead table
  -- NOTE: Add owner_user_id to silver_lead SELECT if not already present.
  -- slu.full_name AS owner_full_name,
  -- slu.department AS owner_department,
  -- slu.is_active AS owner_is_active,

  -- Converted Account Fields (Joined)
  ca.account_name AS converted_account_name,
  ca.industry AS converted_account_industry,
  ca.account_type AS converted_account_type,
  ca.is_life_science_key_account AS converted_account_is_lsk

FROM live.silver_lead sl
-- LEFT JOIN live.silver_user slu ON sl.owner_user_id = slu.user_id -- Uncomment if Owner ID is available in silver_lead
LEFT JOIN live.silver_account ca ON sl.converted_account_id = ca.account_id;


In [0]:
--=========================================================
-- Gold Sales Pipeline Aggregate Table
--=========================================================
-- Purpose: Provide summarized metrics for pipeline analysis over time, by various dimensions.
--          including owner's manager, owner, forecast category, opportunity type, etc.
-- Grain: Close Month, Owner Manager, Owner, Forecast Category, Opportunity Type (adjust dimensions as needed).
--=========================================================

CREATE OR REFRESH LIVE TABLE gold_sales_pipeline_agg (
  CONSTRAINT close_month_not_null EXPECT (close_month IS NOT NULL)
)
COMMENT "Monthly aggregated sales pipeline metrics based on opportunity close date, includes owner manager dimension."
TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  -- Dimensions
  DATE_TRUNC('MONTH', close_date) AS close_month,
  owner_manager_full_name, -- Group by manager name
  owner_full_name,
  owner_department,
  forecast_category,
  opportunity_type,
  account_industry,
  account_billing_country,

  -- Aggregated Metrics
  COUNT(DISTINCT opportunity_id) AS count_total_closed_opportunities,
  COUNT(DISTINCT CASE WHEN is_closed_won = true THEN opportunity_id ELSE NULL END) AS count_closed_won_opportunities,
  COUNT(DISTINCT CASE WHEN is_closed_lost = true THEN opportunity_id ELSE NULL END) AS count_closed_lost_opportunities,

  SUM(CASE WHEN is_closed_won = true THEN amount ELSE 0 END) AS sum_closed_won_amount,
  SUM(CASE WHEN is_closed_lost = true THEN amount ELSE 0 END) AS sum_closed_lost_amount,

  -- Rounding average days
  ROUND(AVG(CASE WHEN is_closed_won = true THEN sales_cycle_days ELSE NULL END), 1) AS avg_won_sales_cycle_days,

  -- Calculated Rates (Calculation logic is the same, using safe division)
  ROUND(
    COALESCE(
      SUM(CASE WHEN is_closed_won = true THEN 1.0 ELSE 0.0 END) -- Use 1.0 for floating point division
      / NULLIF(SUM(CASE WHEN is_closed = true THEN 1 ELSE 0 END), 0), -- Avoid division by zero
      0.0 -- Default to 0
    ),
  4) AS win_rate_by_count, -- Rounded to 4 decimal places

  ROUND(
    COALESCE(
      SUM(CASE WHEN is_closed_won = true THEN amount ELSE 0 END)
      / NULLIF(SUM(CASE WHEN is_closed = true THEN amount ELSE 0 END), 0), -- Avoid division by zero
      0.0 -- Default to 0
    ),
  4) AS win_rate_by_amount -- Rounded to 4 decimal places

FROM live.gold_opportunity_snapshot -- Source now includes manager information
WHERE is_closed = true -- Aggregate based on closed opportunities
GROUP BY
  close_month,
  owner_manager_full_name, -- Group by manager
  owner_department,
  owner_full_name,
  forecast_category,
  opportunity_type,
  account_industry,
  account_billing_country;


In [0]:
--=========================================================
-- Gold Lead Source Aggregate Table
--=========================================================
-- Purpose: Summarize lead generation and conversion performance by source and other dimensions.
-- Grain: Creation Month, Lead Source, Lead Type (adjust dimensions as needed).
--=========================================================

CREATE OR REFRESH LIVE TABLE gold_lead_source_agg (
   CONSTRAINT creation_month_not_null EXPECT (creation_month IS NOT NULL)
)
COMMENT "Monthly aggregated lead performance metrics based on lead creation date."
TBLPROPERTIES ("quality" = "gold")
AS
SELECT
  -- Dimensions
  DATE_TRUNC('MONTH', lead_created_datetime) AS creation_month,
  lead_source,
  lead_type,
  lead_industry,
  lead_country,

  -- Aggregated Metrics
  COUNT(DISTINCT lead_id) AS count_created_leads,
  COUNT(DISTINCT CASE WHEN is_converted = true THEN lead_id ELSE NULL END) AS count_converted_leads,

  AVG(CASE WHEN is_converted = true THEN days_to_convert ELSE NULL END) AS avg_days_to_convert,
  AVG(lead_score) AS avg_lead_score_created,
  AVG(CASE WHEN is_converted = true THEN lead_score ELSE NULL END) AS avg_lead_score_converted,
  AVG(pardot_score) AS avg_pardot_score_created,
  AVG(CASE WHEN is_converted = true THEN pardot_score ELSE NULL END) AS avg_pardot_score_converted,

  -- Calculated Rates (using safe division concept)
   COALESCE(
     SUM(CASE WHEN is_converted = true THEN 1 ELSE 0 END) / NULLIF(COUNT(DISTINCT lead_id), 0),
     0
  ) AS conversion_rate

FROM live.gold_lead_funnel
GROUP BY
  DATE_TRUNC('MONTH', lead_created_datetime),
  lead_source,
  lead_type,
  lead_industry,
  lead_country;